# Задание 1

In [7]:
import numpy as np
import pickle
import string
import tensorflow as tf
from collections import Counter
from pymorphy2 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
punctuation = string.punctuation + '—«»…'
morph = MorphAnalyzer()

In [8]:
def preprocess(text):
    tokens = [token.strip(punctuation) for token in text.lower().split()]
    return [morph.parse(word)[0].normal_form for word in tokens if word] # добавляем лемматизацию

In [9]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(
                   embeddings[word2id[word]].reshape(1, -1), 
                   embeddings).argsort()[0][:10]]
    return similar

In [ ]:
vocab = Counter()
wiki = open('wiki_data.txt', encoding='utf-8-sig')
preprocessed_texts = [preprocess(text) for text in wiki]
with open('preprocessed_texts.pickle', 'wb') as f:
    pickle.dump(preprocessed_texts, f)
for text in preprocessed_texts:
    vocab.update(text)
filtered_vocab = [word for word in vocab if vocab[word] > 30]
word2id = {word: order + 1 for order, word in enumerate(filtered_vocab)}
word2id['PAD'] = 0
id2word = {i: word for word, i in word2id.items()}
sentences = []
for text in preprocessed_texts:
    ids = [word2id[token] for token in text if token in word2id]
    sentences.append(ids)
vocab_size = len(id2word)
sum_occurences = sum(vocab[i] for i in filtered_vocab)
word_probs = [vocab[word] / sum_occurences for word in filtered_vocab] # вероятности для выбора негативных примеров

## Skip-gram

In [9]:
def gen_batches_skip_gram(sentences, window, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent) - 1):
                word = sent[i]
                context = (sent[max(0, i - window // 2):i] + 
                           sent[i + 1:i + window // 2]) # окно разбивается на две половины
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(word2id[np.random.choice(
                        filtered_vocab, p=word_probs)]) # негативные примеры выбираются пропорционально вероятностям
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [26]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))
embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )
target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)
sg_model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
sg_model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [27]:
sg_model.fit(gen_batches_skip_gram(sentences, window=12), # размер окна изменён
             validation_data=gen_batches_skip_gram(sentences,  window=12),
             batch_size=1000,
             steps_per_epoch=2000,
             validation_steps=30,
             epochs=2)

Epoch 1/2
2000/2000 [==============================] - 5339s 3s/step - loss: 0.6816 - accuracy: 0.5472 - val_loss: 0.6712 - val_accuracy: 0.5910
Epoch 2/2
2000/2000 [==============================] - 5878s 3s/step - loss: 0.6689 - accuracy: 0.5745 - val_loss: 0.6703 - val_accuracy: 0.5894


In [33]:
sg_embeddings = sg_model.layers[2].get_weights()[0]
sg_embeddings = 
with open('sg_embeddings.pickle', 'wb') as f:
    pickle.dump(sg_embeddings, f)

## CBOW

In [45]:
def gen_batches_cbow(sentences, window, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = (sent[max(0, i - window // 2):i] + 
                           sent[i + 1:i + window // 2]) # окно разбивается на две половины

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(word2id[np.random.choice(
                    filtered_vocab, p=word_probs)])  # негативные примеры выбираются пропорционально вероятностям
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [50]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))
embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )
target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)
model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
model.fit(gen_batches_cbow(sentences, window=6), # размер окна изменён
          validation_data=gen_batches_cbow(sentences,  window=6),
          batch_size=1000,
          steps_per_epoch=2000,
          validation_steps=30,
          epochs=2)

Epoch 1/2
2000/2000 [==============================] - 4921s 2s/step - loss: 0.6865 - accuracy: 0.5500 - val_loss: 0.6738 - val_accuracy: 0.6005
Epoch 2/2
2000/2000 [==============================] - 4733s 2s/step - loss: 0.6553 - accuracy: 0.6206 - val_loss: 0.6328 - val_accuracy: 0.6441


In [54]:
cbow_embeddings = model.layers[2].get_weights()[0]
with open('cbow_embeddings.pickle', 'wb') as f:
    pickle.dump(cbow_embeddings, f)

## Сравнение

In [65]:
def compare(word):
    print(word)
    print(f'Skip-gram: {most_similar(word, sg_embeddings)}\nCBOW: {most_similar(word, cbow_embeddings)}')

In [66]:
compare('январь')

январь
Skip-gram: ['январь', '2019', 'июнь', 'ноябрь', 'февраль', '2011', 'март', 'октябрь', '23', '1890']
CBOW: ['январь', 'апрель', 'июнь', 'май', 'декабрь', 'октябрь', 'сентябрь', 'июль', 'март', '2009']


In [71]:
compare('война')

война
Skip-gram: ['война', 'отечественный', 'мировой', 'великий', 'армия', 'второй', 'пехотный', 'париж', '16-й', 'ход']
CBOW: ['война', '1788', 'вступить', 'отечественный', '1978', 'русско-японский', '1994', 'мировой', '2003', '2004']


In [91]:
compare('город')

город
Skip-gram: ['город', 'центр', 'старое', 'провинция', 'долина', 'панихида', 'район', 'пункт', 'община', 'слияние']
CBOW: ['город', 'переименовать', 'состав', 'поступить', 'частность', 'вступить', 'район', 'включить', 'участвовать', 'расположить']


In [100]:
compare('море')

море
Skip-gram: ['море', 'побережье', 'юг', 'северо-восточный', 'северный', 'центральный', 'север', 'восточный', 'юго-восточный', 'архангельский']
CBOW: ['море', 'находиться', 'рука', 'км', 'fanuc', 'качество', 'восток', 'часть', 'paphiopedilum', 'борьба']


Похоже, что skip-gram модель генерирует более правдоподобные эмбеддинги, вероятно, из-за большего размера окна

# Задание 2

In [103]:
import gensim

In [148]:
%%time
word2vec_model = gensim.models.Word2Vec(preprocessed_texts,
                                       vector_size=200, # этого вполне достаточно
                                       min_count=0,
                                       max_vocab_size=400000, # мы не хотим упускать никакие слова
                                       window=4,
                                       epochs=10, # нормальный баланс между качеством и скоростью
                                       hs=1,
                                       negative=0,
                                       sample=1e-4,
                                       ns_exponent=0,
                                       cbow_mean=0)

Wall time: 1min 59s


In [151]:
%%time
fasttext_model = gensim.models.FastText(preprocessed_texts,
                                        vector_size=200, 
                                        min_count=0,
                                        max_vocab_size=400000, 
                                        window=4,
                                        epochs=10,
                                        hs=1,
                                        negative=0,
                                        sample=1e-4,
                                        ns_exponent=0,
                                        cbow_mean=0,
                                        min_n=4,
                                        max_n=7)

Wall time: 12min 36s


In [153]:
with open('fasttext_model.pickle', 'wb') as f:
    pickle.dump(fasttext_model,f)

In [154]:
word2vec_model.wv.most_similar('процессор')

[('интерфейс', 0.6308376789093018),
 ('встроить', 0.5643938779830933),
 ('api', 0.5449751019477844),
 ('виртуальный', 0.5367855429649353),
 ('контроллер', 0.5338598489761353),
 ('core', 0.5334622263908386),
 ('ibm', 0.5180827975273132),
 ('клавиатура', 0.5154924392700195),
 ('файл', 0.5106635689735413),
 ('микроархитектура', 0.5063623189926147)]

In [155]:
fasttext_model.wv.most_similar('процессор')

[('препроцессор', 0.9551441073417664),
 ('сопроцессор', 0.953107476234436),
 ('xinclude-процессор', 0.9361088275909424),
 ('кмоп-микропроцессор', 0.9199901819229126),
 ('микропроцессор', 0.905845046043396),
 ('видеопроцессор', 0.8963738083839417),
 ('platform.############процессор', 0.8771792054176331),
 ('процессорный', 0.8616329431533813),
 ('directx-видеопроцессор', 0.8447694778442383),
 ('наоборот.############процессор', 0.824285089969635)]

In [179]:
word2vec_model.wv.most_similar('ракета')

[('самолёт', 0.6735202074050903),
 ('вертолёт', 0.5849144458770752),
 ('реактивный', 0.5836101174354553),
 ('корабль', 0.5830175280570984),
 ('ракетный', 0.571421205997467),
 ('модификация', 0.5659030079841614),
 ('снаряд', 0.5382475852966309),
 ('истребитель', 0.530783474445343),
 ('авианосец', 0.5228886008262634),
 ('машина', 0.522082507610321)]

In [180]:
fasttext_model.wv.most_similar('ракета')

[('ракета-торпеда', 0.8745078444480896),
 ('снаряд-ракета', 0.858798086643219),
 ('каракета', 0.8372753858566284),
 ('ракетчик', 0.8151965737342834),
 ('потакета', 0.8082267045974731),
 ('ракетоплан', 0.7970840334892273),
 ('ракето-торпеда', 0.7814498543739319),
 ('ракета-носитель', 0.7596801519393921),
 ('ракетоносец', 0.7411296367645264),
 ('ракета.############разработать', 0.7398077845573425)]